In [25]:
# 크롤링 패키지 import
import requests
from bs4 import BeautifulSoup

In [26]:
# 워드클라우드 패키지 import
from collections import Counter
from string import punctuation
from wordcloud import WordCloud
from eunjeon import Mecab
import base64
mecab = Mecab()

In [27]:
def count_word_frequency(data):
    _data = data.lower()
    
    for i in punctuation:
        _data = _data.replace(i, "")
    
    noun = mecab.nouns(_data)
    counter = Counter(noun)
    
    return counter

def create_word_cloud(data):
    counter = count_word_frequency(data)
    
    cloud = WordCloud(font_path="‪‪C:\Windows\Fonts\malgun.ttf", background_color="white")
    cloud.fit_words(counter)
    cloud.to_file('test_wc2.png')

In [28]:
def crawling(soup):
    result = ""
    for children in soup.find("div", class_="_article_body_contents").children:
        if children.name == None:
            result += children
    start = result.find("// TV플레이어")
    result = result[start + len("// TV플레이어")+1: ]
    end = result.find("// 본문 내용")
    result = result[:end]
    
    return result.replace("\n", "").replace("\t", "")

In [29]:
def get_href(soup):
    res = []
    cluster_body = soup.find("div", class_="cluster_body")
    
    for cluster_text in cluster_body.find_all("div", class_="cluster_text"):
        res.append(cluster_text.find("a")["href"])
        
    return res

In [30]:
def get_request(section):
    url = "https://news.naver.com/main/main.nhn"
    sec_dict = {
        "정치": 100,
        "경제": 101,
        "사회": 102,
        "생활": 103,
        "세계": 104,
        "과학": 105
    }
    return requests.get(url, params={"sid1":sec_dict[section]})

In [31]:
def main():
    list_href = []
    res = []
    
    section = input('"정치", "경제", "사회", "생활", "세계", "과학" 중 하나를 입력하세요. \n')
    
    req = get_request(section)
    soup = BeautifulSoup(req.text, "html.parser")
    
    list_href = get_href(soup)
    
    for href in list_href:
        href_req = requests.get(href)
        href_soup = BeautifulSoup(href_req.text, "html.parser")
        res.append(crawling(href_soup))
    
    text = " ".join(res)
    create_word_cloud(text)

In [32]:
if __name__ == "__main__":
    main()

"정치", "경제", "사회", "생활", "세계", "과학" 중 하나를 입력하세요. 
정치
